# Robot playground

## 1. Parameters and Helper function

In [ ]:
import os
import subprocess
import ipywidgets as widgets
from IPython.display import display, Markdown

LAUNCH_DIR = os.path.abspath(os.path.join(os.getcwd(), "../launch"))

try:
    HAS_DISPLAY = 'true' if os.environ['DISPLAY'] != ':100' else 'false'
except KeyError:
    HAS_DISPLAY = 'false'

def launch_robot(robot):
    launchfile = os.path.join(LAUNCH_DIR, f"{robot}_standalone.launch")
    command = f"roslaunch {launchfile} gui:={HAS_DISPLAY}"
    # print(command)
    !$command
    # return subprocess.Popen(command, shell=True)

## 2. Launch one of the following robot

>Note: The launch process will run in foreground and block the execution of the the following code cells!

In [ ]:
ROBOT_LIST = ['pr2', 'tiago', 'hsr', 'donbot', 'tracy', 'boxy', 'stretch', 'armar']
buttons = [widgets.Button(description=robot) for robot in ROBOT_LIST]

for button in buttons:
    button.on_click(lambda b, robot=button.description: launch_robot(robot))

button_box = widgets.HBox(buttons)
display(Markdown("## Click the buttons to launch a robot!"))
display(button_box)

## Todos:
- Launch robot in a background subprocess. 
- Add Giskard Python Interface demo
- Blockly control
- Fix boxy arms are not visible in Rvizweb
- 
- TF info is not init for some robots (tiago, donbot, stretch under giskard:devel)
- Rvizweb scene refresh